#### Imports

In [1]:
import datetime
import pandas as pd
import hopsworks
from functions import util
import os

/opt/hostedtoolcache/Python/3.12.8/x64/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-01-22 01:12:33,381 INFO: Failed to extract font properties from /usr/share/fonts/truetype/noto/NotoColorEmoji.ttf: In FT2Font: Can not load face (unknown file format; error code 0x2)


2025-01-22 01:12:33,488 INFO: generated new fontManager


In [2]:
with open('../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store() 

country="Sweden"
city = "Stockholm"
latitude = "59.3294"
longitude = "18.0687"

today = datetime.date.today()

2025-01-22 01:12:33,834 INFO: Initializing external client


2025-01-22 01:12:33,834 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-01-22 01:12:34,686 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207502


#### Get references to the Feature Groups </span>

In [3]:
# Retrieve feature groups
accidents_fg = fs.get_feature_group(
    name='accidents',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

#### Get Weather Forecast data</span>

In [4]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df['day_of_week'] = daily_df['date'].dt.dayofweek

daily_df

Coordinates 59.25°N 18.0°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,day_of_week
0,2025-01-22,1.75,0.0,11.681987,213.690094,Stockholm,2
1,2025-01-23,2.20,0.2,17.654688,129.207199,Stockholm,3
2,2025-01-24,3.00,0.0,14.512064,209.744797,Stockholm,4
3,2025-01-25,6.55,0.0,26.478971,226.101624,Stockholm,5
4,2025-01-26,1.95,0.0,6.379216,196.389618,Stockholm,6
5,2025-01-27,2.25,0.0,12.574260,166.759476,Stockholm,0
6,2025-01-28,5.20,0.0,10.853866,185.710495,Stockholm,1
7,2025-01-29,5.05,0.0,16.766108,194.931473,Stockholm,2
8,2025-01-30,5.00,0.1,12.574260,203.629395,Stockholm,3
9,2025-01-31,5.35,0.0,19.174856,219.667801,Stockholm,4


In [5]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
 6   day_of_week                  10 non-null     int32         
dtypes: datetime64[ns](1), float32(4), int32(1), object(1)
memory usage: 492.0+ bytes


In [6]:
weather_fg.insert(daily_df)

Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 40.00% |████      | Rows 4/10 | Elapsed Time: 00:01 | Remaining Time: 00:01

Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00

Launching job: weather_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207502/jobs/named/weather_1_offline_fg_materialization/executions


(Job('weather_1_offline_fg_materialization', 'SPARK'), None)